In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test2': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {source: datasets.ImageFolder(source, transform)
                    for (source, transform) in data_transforms.items()}
data_loaders = {source: torch.utils.data.DataLoader(image_dataset, batch_size=4, shuffle=True, num_workers=4)
                    for (source, image_dataset) in image_datasets.items()}

dataset_sizes = {source: len(image_dataset) for (source, image_dataset) in image_datasets.items()}
class_names = image_datasets['train'].classes